## 1.Загрузка данных и подготовка к анализу

##### изучение данных <a name="1.1-bullet"></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df_v= pd.read_csv('/datasets/visits_log.csv')
df_v.describe()

In [ ]:
df_v.info()

In [ ]:
df_v.head(10)

Необходимо преобразовать столбцы с датами в соответствующий тип

In [ ]:
df_v['End Ts'] = pd.to_datetime(df_v['End Ts'])
df_v['Start Ts'] = pd.to_datetime(df_v['Start Ts'])

In [ ]:
df_v.info()

In [ ]:
df_o= pd.read_csv('/datasets/orders_log.csv')

In [ ]:
df_o.info()

In [ ]:
df_o.head(10)

Необходимо преобразовать столбцы с датами в соответствующий тип

In [ ]:
df_o['Buy Ts']=pd.to_datetime(df_o['Buy Ts'])

In [ ]:
df_o.info()

In [ ]:
df_с= pd.read_csv('/datasets/costs.csv')

In [ ]:
df_с.info()

In [ ]:
df_с.head(10)

Необходимо преобразовать столбцы с датами в соответствующий тип

In [ ]:
df_с['dt']=pd.to_datetime(df_с['dt'])

In [ ]:
df_с.info()

###### поиск дубликатов <a name="1.2-bullet"></a>

In [ ]:
df_с.duplicated().sum()

In [ ]:
df_o.duplicated().sum()

In [ ]:
df_o.duplicated().sum()

###### сравнение временных промежутков  <a name="1.3-bullet"></a>

In [ ]:
df_o['Buy Ts'].min()

In [ ]:
df_o['Buy Ts'].max()

In [ ]:
df_с['dt'].min()

In [ ]:
df_с['dt'].max()

In [ ]:
df_v['End Ts'].min()

In [ ]:
df_v['Start Ts'].min()

In [ ]:
df_v['End Ts'].max()

In [ ]:
df_v['Start Ts'].max()

#### Вывод <a name="1.4-bullet"></a>

Дубликатов нет, временные промежутки совпадают.
Данные пригодны для анализа. 

In [ ]:
df_v.info()

In [ ]:
df_v[df_v['Start Ts']>df_v['End Ts']]

In [ ]:
df_v=df_v[df_v['Start Ts']<df_v['End Ts']]

In [ ]:
df_o.info()

In [ ]:
df_o[df_o['Revenue']<=0]

In [ ]:
df_o=df_o[df_o['Revenue']>0]

## 2.Постройка отчетов и расчет метрик: продукты 

###### Сколько людей пользуются в день, неделю, месяц? <a name="2.1-bullet"></a>

In [ ]:
#группировка по дню и поиск уникальных пользователей
df_o['day']=df_o['Buy Ts'].dt.day
day_u=df_o.groupby('day').agg({'Uid': 'nunique'})
day_u.mean()

In [ ]:
df_v['day'] = df_v['Start Ts'].astype('datetime64[D]')
day_u=df_v.groupby('day').agg({'Uid': 'nunique'})

In [ ]:
df_v['day']

In [ ]:
day_u.reset_index(inplace=True)

In [ ]:
day_u.columns=['day','Uid']

In [ ]:
fig = px.line(day_u, x = 'day', y = 'Uid', title = 'количество пользователей в день')
fig.show()

In [ ]:
day_u_d=df_v.groupby(['day','Device']).agg({'Uid': 'nunique'}).reset_index()
fig = px.line(day_u_d, x = 'day',color='Device', y = 'Uid', title = 'количество пользователей в день по устройствам')
fig.show()

Графики с количеством пользователей в день в зависимости от устройства  совпадают по динамике с общим графиком посещений в день. Можно сказать, что количество посещений в день не зависит от устройства. 

In [ ]:
#группировка по неделе и поиск уникальных пользователей 
df_o['week'] = df_o['Buy Ts'].dt.week
week_u = df_o.groupby('week').agg({'Uid': 'nunique'})

In [ ]:
week_u.mean()

In [ ]:
#определение количества уникальных пользователей в неделю
df_v['Year-Week']= df_v['Start Ts'].astype('datetime64[W]')
week_u=df_v.groupby('Year-Week').agg({'Uid': 'nunique'})

In [ ]:
week_u

In [ ]:
week_u=week_u.reset_index()

In [ ]:
fig = px.line(week_u, x="Year-Week", y="Uid",title="количество пользователей в неделю")
fig.show()

In [ ]:
week_u=df_v.groupby('Year-Week').agg({'Uid': 'nunique'})

In [ ]:
week_u_d=df_v.groupby(['Year-Week','Device']).agg({'Uid': 'nunique'})
week_u_d=week_u_d.reset_index()

In [ ]:
fig = px.line(week_u_d, x="Year-Week", y="Uid",color='Device',title="количество пользователей в неделю в зависимости от устройства")
fig.show()

Динамика количества пользователей с компьютера совпадает со средней динамикой, пользователи с телефона имеют более стабильную динамику посещений с октября по март. 

In [ ]:
#группировка по месяцу и поиск уникальных пользователей
df_o['month']=df_o['Buy Ts'].dt.month
month_u = df_o.groupby('month').agg({'Uid': 'nunique'})

In [ ]:
month_u.mean()

In [ ]:
df_v['month'] =df_v['Start Ts'].astype('datetime64[M]')

In [ ]:
month_u = df_v.groupby('month').agg({'Uid': 'nunique'}).reset_index()

In [ ]:
fig = px.line(month_u, x="month", y="Uid",title="количество пользователей в месяц")
fig.show()

In [ ]:
month_u_d = df_v.groupby(['month','Device']).agg({'Uid': 'nunique'}).reset_index()
fig = px.line(month_u_d, x="month",color='Device', y="Uid",title="количество пользователей в месяц в зависимости от устройства")
fig.show()

Динамика количества пользователей с компьютера совпадает со средней динамикой, пользователи с телефона имеют более стабильную динамику посещений с октября по март .

###### Сколько сессий в день? <a name="2.2-bullet"></a>

In [ ]:
df_v.info()

In [ ]:
df_v

In [ ]:
#расчет общего количества сессий на сайте
df_v['day'] = df_v['End Ts'].astype('datetime64[D]')
events_count = df_v.groupby('day').agg({'Uid': 'count'}).reset_index()
events_count.mean()

In [ ]:
#расчет количества сессий у каждого пользователя
events_count_user = df_v.groupby('Uid')[['day']].count()
events_count_user.mean()

In [ ]:
events_count

In [ ]:
fig = px.line(events_count, x="day", y="Uid",title="количество сессий в день в динамике")
fig.show()

In [ ]:
#расчет количества сессий в зависимости от устройства
events_count_d = df_v.groupby(['day','Device']).agg({'Uid': 'count'}).reset_index()
fig = px.line(events_count_d, x="day",color='Device', y="Uid",title="количество сессий в день в динамике в зависимости от устройства")
fig.show()

Количество пользователей с телефона имеет более ровную динамику и меньше совпадает с общей динамикой для обоих устройств. 

In [ ]:
events_count_d

###### Сколько длится одна сессия? <a name="2.3-bullet"></a>

In [ ]:
df_v['duration']=df_v['End Ts']-df_v['Start Ts']
df_v['duration'].mean()

In [ ]:
df_v['duration'].median()

In [ ]:
df_v['duration'].mode()

In [ ]:
fig = px.histogram(df_v, x = 'duration',
                  title = 'Распределение продолжительности одной сессии')
fig.show()

In [ ]:
fig = px.histogram(df_v, x = 'duration', color='Device',
                  title = 'Распределение продолжительности одной сессии')
fig.show()

- график представляет из себя распределение Пуассона
- Как правило одна сессия длится около трех минут
- Люди за компьютером сидят дольше, чем с телефона

##### Как часто люди возвращаются? <a name="2.4-bullet"></a>

In [ ]:
df_v.info()

In [ ]:
#формирование когорт
first_activity_date = df_v.groupby(['Uid'])['Start Ts'].min()
first_activity_date.name = 'first_visit_date'
user_activity = df_v.join(first_activity_date,on='Uid')
user_activity['first_visit_date'] = user_activity['first_visit_date'].astype('datetime64[M]')

In [ ]:
def int_r(num):
    num = int(num + (0.5 if num > 0 else -0.5))
    return num

In [ ]:
user_activity['Start_ts_month'] = user_activity['Start Ts'].astype('datetime64[M]')
user_activity['cohort_lifetime'] = user_activity['Start_ts_month'] - user_activity['first_visit_date']
user_activity['cohort_lifetime'] = user_activity['cohort_lifetime'] / np.timedelta64(1,'M')
user_activity['cohort_lifetime'] = user_activity['cohort_lifetime'].apply(int_r)
user_activity['cohort_lifetime'] = user_activity['cohort_lifetime'].astype('int') 


In [ ]:
cohorts = user_activity.groupby(['first_visit_date','cohort_lifetime']).agg({'Uid':'nunique'}).reset_index()
initial_users_count = cohorts[cohorts['cohort_lifetime'] == 0][['first_visit_date','Uid']]
initial_users_count = initial_users_count.rename(columns={'Uid':'cohort_users'})
cohorts = cohorts.merge(initial_users_count,on='first_visit_date')
cohorts['retention'] = cohorts['Uid']/cohorts['cohort_users']
retention_pivot = cohorts.pivot_table(index='first_visit_date',columns='cohort_lifetime',values='retention',aggfunc='sum')

In [ ]:
sns.set(style='white')
plt.figure(figsize=(22, 20))
plt.title('Cohorts: User Retention')
sns.heatmap(retention_pivot, annot=True, fmt='.1%', linewidths=1, linecolor='gray',vmin=0,vmax=0.15); 


Качество когорт с течением времени улучшается, все больше посетителей возвращается на сайт. Это может быть связано с улучшением качества источников трафика, улучшением качества самого сервиса.

In [ ]:
user_activity.info()

In [ ]:
sour_pv_d=user_activity.pivot_table(index=['Uid'],columns='Device',values='Start Ts',aggfunc='min').reset_index()
sour_pv_d

In [ ]:
dev_pv=df_v.pivot_table(index=['Uid','Device'],values='Start Ts',aggfunc='min').reset_index()

In [ ]:
dev_pv=dev_pv.drop(['Start Ts'], axis=1)
dev_pv.columns=['Uid','main_device']
dev_pv

In [ ]:
sour_pv_d.loc[~(pd.isnull(sour_pv_d['desktop'])),'main_device']='desktop'
sour_pv_d.loc[~(pd.isnull(sour_pv_d['touch'])),'main_device']='touch'

In [ ]:
sour_pv_d=sour_pv_d.drop(['desktop', 'touch'], axis=1)

In [ ]:
sour_pv_d.info()

In [ ]:
user_activity.info()

In [ ]:
user_activity=user_activity.merge(dev_pv,on='Uid',how='left')

In [ ]:
cohorts.sample()

In [ ]:
user_activity.info()

In [ ]:
initial_users_count

In [ ]:
cohorts_d = user_activity.groupby(['first_visit_date','cohort_lifetime','main_device']).agg({'Uid':'nunique'}).reset_index()

In [ ]:
cohorts_d_d=cohorts_d[cohorts_d['main_device']=='desktop']

In [ ]:
initial_users_count = cohorts_d_d[(cohorts_d_d['cohort_lifetime'] == 0)][['first_visit_date','Uid',]]
initial_users_count = initial_users_count.rename(columns={'Uid':'cohort_users'})
cohorts_d_d = cohorts_d_d.merge(initial_users_count,on='first_visit_date')
cohorts_d_d['retention'] = cohorts_d_d['Uid']/cohorts_d_d['cohort_users']
#retention_pivot = cohorts_d.pivot_table(index=['first_visit_date','main_device'],columns='cohort_lifetime',values='retention',aggfunc='sum')
retention_pivot_d = (cohorts_d_d.
                     pivot_table(index='first_visit_date',columns='cohort_lifetime',values='retention',aggfunc='sum'))

In [ ]:
initial_users_count

In [ ]:
cohorts_d

In [ ]:
cohorts_d.info()

In [ ]:
sns.set(style='white')
plt.figure(figsize=(22, 20))
plt.title('Cohorts: User Retention for desktop')
sns.heatmap(retention_pivot_d, annot=True, fmt='.1%', linewidths=1, linecolor='gray',vmin=0,vmax=0.15); 

In [ ]:
cohorts_d_t=cohorts_d[cohorts_d['main_device']=='touch']

In [ ]:
initial_users_count = cohorts_d_t[(cohorts_d_t['cohort_lifetime'] == 0)][['first_visit_date','Uid',]]
initial_users_count = initial_users_count.rename(columns={'Uid':'cohort_users'})
cohorts_d_t = cohorts_d_t.merge(initial_users_count,on='first_visit_date')
cohorts_d_t['retention'] = cohorts_d_t['Uid']/cohorts_d_t['cohort_users']
#retention_pivot = cohorts_d.pivot_table(index=['first_visit_date','main_device'],columns='cohort_lifetime',values='retention',aggfunc='sum')
retention_pivot_t = (cohorts_d_t.
                     pivot_table(index='first_visit_date',columns='cohort_lifetime',values='retention',aggfunc='sum'))

In [ ]:
sns.set(style='white')
plt.figure(figsize=(22, 20))
plt.title('Cohorts: User Retention for touch')
sns.heatmap(retention_pivot_t, annot=True, fmt='.1%', linewidths=1, linecolor='gray',vmin=0,vmax=0.15); 

Retantion Rate не меняется в зависимости от устройства

In [ ]:
cohorts

In [ ]:
#группировка по количеству пользователей и подсчет количества покупок 
events_return = df_o.groupby('Uid').agg({'Buy Ts': 'count'})
events_return.mean()

### Вывод <a name="2.5-bullet"></a>

- В день сервисом пользуется 1437 человек, в неделю - 849, в месяц - 3721. То, что в неделю уникальных посетителей меньше, чем в день, можно объяснить тем, что пользователи, планируя что-то купить, могут заходить повторно в течение недели, так уникальные пользователи для каждого дня не уникальны для недели, что влияет на значения. Большое количество пользователей в месяц связано с большим временным диапазоном. 
- Количество сессий в день в среднем 11594. Длина сессии - 11 минут.
- Нередко пользователи совершают заказ более одного раза и довольно часто заходят на сайт повторно

## 3.Постройка отчетов и расчет метрик: продажи

Необходимо выделить тех пользователей, что совершили заказы и определить, во время какой по счету сессии они совершили заказ (или заказы). 

In [ ]:
u_buy=df_v[df_v['Uid'].isin(df_o['Uid'])]

#### составление когорт по времени первого заказа

In [ ]:
df_o.info()

In [ ]:
# группировка по уникальному  номеру пользователя, с поиском первой даты покупки
first_time = df_o.groupby('Uid')['Buy Ts'].min() 

In [ ]:
first_time.name = 'first_time_buy'
# обращаемся к атрибуту name Series в переменной first_order_date_by_customers

In [ ]:
first_time

In [ ]:
#добавление столбца с первой покупкой
u_buy = u_buy.join(first_time,on='Uid')
print(u_buy.head(10))

##### Когда люди начинают покупать? <a name="3.1-bullet"></a>

Поделим людей на 2 категории:
1. Количество тех, кто совершил первую покупку не в первый визит
2. Количество тех, кто совершил покупку в первый визит

In [ ]:
type_1= u_buy[u_buy['first_time_buy']<u_buy['Start Ts']].shape[0]
type_2= u_buy[u_buy['first_time_buy']==u_buy['Start Ts']].shape[0]
print('количество тех, кто совершает покупку не в первый визит',type_1)
print('количество тех, кто совершает покупку в первый визит',type_2)

Чаще всего люди совершают покупку во время второго захода на сайт( в среднем, пользователи не заходят на сайт больше одного раза)

In [ ]:
df_o

In [ ]:
b_start=df_o.merge(first_time,on='Uid')

In [ ]:
b_start=b_start.merge(dev_pv,on='Uid')

In [ ]:
b_start

In [ ]:
df_v

In [ ]:
min_event_datetime = df_v.groupby(['Uid'])['Start Ts'].min()
min_event_datetime.name = 'min_event_datetime'
b_start = b_start.join(min_event_datetime,on='Uid')

In [ ]:
b_start=b_start[b_start['first_time_buy']>=b_start['min_event_datetime']]

In [ ]:
b_start['time_purchase']=b_start['first_time_buy']-b_start['min_event_datetime']

In [ ]:
b_start['time_purchase'].median()

In [ ]:
b_start['time_purchase'] = b_start['time_purchase'] / np.timedelta64(1,'D')

In [ ]:
b_start['time_purchase']

In [ ]:
b_start=b_start[b_start['time_purchase']>0]

In [ ]:
b_start

In [ ]:
fig = px.histogram(b_start, x = 'time_purchase',
                  title = 'Распределение продолжительности времени между первым посещением и первой покупкой')
fig.show()

In [ ]:
fig = px.histogram(b_start, x = 'time_purchase',color='main_device',
                  title = 'Распределение продолжительности времени между первым посещением и первой покупкой')
fig.show()

Основная часть пользователей совершает первую покупку в течение первого дня, когда они оказались на сайте. Преобладающая часть пользователей делает это в течение 2 дней. Большая часть покупок совершается с компьютеров. 

In [ ]:
b_start.info()

In [ ]:
df_v.info()

In [ ]:
sour_pv_s=df_v.pivot_table(index=['Uid','Source Id'],values='Start Ts',aggfunc='min').reset_index()

In [ ]:
sour_pv_s

In [ ]:
sour_pv_s=sour_pv_s.drop(columns=['Start Ts'])

In [ ]:
print(sour_pv_s)

In [ ]:
sour_pv_s.columns=['Uid','main_sourse']

In [ ]:
b_start=b_start.merge(sour_pv_s,on='Uid')

In [ ]:
b_start.info()

In [ ]:
b_start_s_pv=b_start.pivot_table(index='main_sourse',values='time_purchase',aggfunc='median').reset_index()

In [ ]:
fig = px.bar(b_start_s_pv, x='main_sourse',y = 'time_purchase',
                  title = 'Распределение продолжительности времени между первым посещением и первой покупкой по разным источникам перехода')
fig.show()

Быстрее всего совершают заказы при переходах с 3 4 5, 7 и 10 источников. 

##### Сколько раз покупают за период? <a name="3.2-bullet"></a>

Необходимо посмотреть, сколько раз совершает покупку каждая когорта в месяц

In [ ]:
df_o

In [ ]:
df_o.info()

In [ ]:
first_time

In [ ]:
df_o = df_o.merge(first_time,on='Uid')
df_o['first_order_month'] = df_o['first_time_buy'].astype('datetime64[M]')
df_o['order_month'] = df_o['Buy Ts'].astype('datetime64[M]')

In [ ]:
df_o_pv=df_o.pivot_table(index='first_order_month',
    columns='order_month',
    values='Uid',
    aggfunc='count') 

In [ ]:
plt.figure(figsize=(13, 9))
plt.title('Количество пользователей по когортам')
sns.heatmap(df_o_pv, annot=True, fmt='.1f', linewidths=1, linecolor='gray') 

В среднем каждая когорта совершает 3000 покупок. Разное количество новых пользователей может быть связана с сезонностью (например, летом меньше проводится мероприятий)

###### Какой средний чек? <a name="3.3-bullet"></a>

In [ ]:
df_o.info()

In [ ]:
orders_grouped_by_cohorts = df_o.groupby(['first_order_month','order_month']).agg({'Revenue':'sum','Uid':'nunique'})

In [ ]:
orders_grouped_by_cohorts

In [ ]:
orders_grouped_by_cohorts['revenue_per_user'] = orders_grouped_by_cohorts['Revenue'] / orders_grouped_by_cohorts['Uid'] 

In [ ]:
orders_grouped_by_cohorts

In [ ]:
orders_grouped_by_cohorts_pv=orders_grouped_by_cohorts.pivot_table(index='first_order_month',columns='order_month',values='revenue_per_user',aggfunc='mean')

In [ ]:
plt.figure(figsize=(13, 9))
plt.title('Средний чек покупателей')
sns.heatmap(orders_grouped_by_cohorts_pv, annot=True, fmt='.1f', linewidths=1, linecolor='gray',vmin=0,vmax=40) 

Тепловая карта, в целом, равномерна, не считая поведения когорт за июнь 2017 и за декабрь 2017, там могло зарегестрироваться сразу несколько человек, которым нравится регулярно ходить на какие-то мероприятия. Так же выбиваются поведение сентябрьсвой когорты  в декабре, там могло совпасть так, что сразу много человек сделали заказ или один человек сделал заказ на крупную группу лиц. 

##### Сколько денег приносят? (LTV) <a name="3.4-bullet"></a>

In [ ]:
df_o.info()

In [ ]:
#cоздание отдельной переменной, где хранится месяц первой покупки для каждого покупателя
first_orders = df_o.groupby('Uid').agg({'order_month': 'min'}).reset_index()
first_orders.columns = ['Uid', 'first_order_month']

In [ ]:
#количество новых покупателей в месяц
cohort_sizes = first_orders.groupby('first_order_month').agg({'Uid': 'nunique'}).reset_index()
#first_order_month = pd.merge(df_o, first_orders, on='Uid')
cohorts = df_o.groupby(['first_order_month', 'order_month']).agg({'Revenue': 'sum'}).reset_index()

In [ ]:
cohort_sizes.info()

In [ ]:
cohorts

In [ ]:
report = pd.merge(cohort_sizes, cohorts, on='first_order_month')
print(report.head())

Поскольку маржинальность неизвестна, предположим, что выручка - это и есть валовая прибыль. 

In [ ]:
#расчет возраста когорт
report['age'] = (report['order_month'] - report['first_order_month']) / np.timedelta64(1, 'M')
report['age'] = report['age'].round().astype('int')
report['ltv'] = report['Revenue'] / report['Uid']
result = report.pivot_table(
        index='first_order_month', 
        columns='age', 
        values='ltv', 
        aggfunc='mean')
result=result.cumsum(axis=1).round(2)

In [ ]:
plt.figure(figsize=(13, 9))
plt.title('Сколько приносят денег LTV')
sns.heatmap(result, annot=True, fmt='.1f', linewidths=1, linecolor='gray') 

За исключением частных случаев, основную прибыль когорта дает в месяц своего появлени. Исключения могут быть связаны с совпадениями, когда сразу много людей решили сделать заказ. Доход от одного клиента  LTV не зависит от сезонности. 

### Вывод  <a name="3.5-bullet"></a>

- люди не склонны совершать покупки в свой первый сеанс
- в большинстве своем люди покупают что-то единожды, затем меньшинство возвращается в течение всего года 

## 4.Постройка отчетов и расчет метрик: маркетинг

##### Сколько денег потратили? Всего / на каждый источник / по времени <a name="4.1-bullet"></a>

In [ ]:
df_с

##### сколько было потрачено всего

In [ ]:
df_с['costs'].sum()

##### сколько было потрачено на каждый источник

In [ ]:
df_с_source=df_с.pivot_table(index='source_id',values='costs',aggfunc='sum')
df_с_source

Важно отметить, что не по всем ресурсам перехода есть информация о затратах (допустим, в данном временном промежутке реклама у этих ресурсов не покупалась)

###### сколько тратилось каждый месяц

In [ ]:
df_с['month'] = df_с['dt'].astype('datetime64[M]')

In [ ]:
df_с

In [ ]:
df_с_month=df_с.pivot_table(index=['month'],values='costs',aggfunc='sum').reset_index()

In [ ]:
df_с_month

In [ ]:
fig = px.line(df_с_month, x = 'month', y = 'costs', title = 'изменение расходов на рекламу по месяцам')
fig.show()


In [ ]:
df_с_month_s=df_с.pivot_table(index=['month','source_id'],values='costs',aggfunc='sum').reset_index()
fig = px.line(df_с_month_s, x = 'month',color='source_id', y = 'costs', title = 'изменение расходов на рекламу по месяцам')
fig.show()

In [ ]:
df_с_sourse=df_с.pivot_table(index=['source_id'],values='costs',aggfunc='sum').reset_index()
fig = px.bar(df_с_sourse, x='source_id', y = 'costs', title = 'изменение расходов на рекламу по месяцам')
fig.show()

- Больльше всего тратится на рекламу с августа по апрель
- Самый дорогой источник тот, что под номером 3, самые дешевые - под номерами 9 и 10
- Динамика расходов на рекламу для всех источников одинакова

###### Сколько стоило привлечение одного покупателя из каждого источника? <a name="4.2-bullet"></a>

Необходимо выявить количество покупателей из каждого источника

In [ ]:
df_v.info()

In [ ]:
dev_pv=df_v.pivot_table(index=['Uid','Device'],values='Start Ts',aggfunc='min').reset_index()

In [ ]:
dev_pv=dev_pv.drop(columns=['Start Ts'])

In [ ]:
dev_pv

In [ ]:
dev_pv.columns=['Uid','main_device']

In [ ]:
df_v=df_v.merge(dev_pv,on='Uid',how='left')

###### установление покупателей из каждого  источника перехода

In [ ]:
df_v.info()

In [ ]:
df_v=df_v.merge(sour_pv_s,on='Uid')

In [ ]:
#определение количества покупателей с каждого источника перехода
sourse_count=df_v.pivot_table(index='main_sourse',values='Uid',aggfunc='count')
sourse_count

In [ ]:
sourse_count.reset_index(drop=False, inplace=True)
sourse_count

In [ ]:
sourse_count.columns=['source_id','Uid']

In [ ]:
sourse_count

In [ ]:
df_с_source

In [ ]:
df_с_source.reset_index(drop=False, inplace=True)

In [ ]:
sourse_count.columns=['source_id','Uid']

In [ ]:
sourse_count

In [ ]:
#присоединение к столбцу с расходами столбца с количеством покупателей
source_user = df_с_source.merge(sourse_count,on='source_id')

In [ ]:
df_с_source

In [ ]:
source_user=source_user.fillna(0)

In [ ]:
source_user

In [ ]:
source_user['cac'] = source_user['costs'] / source_user['Uid']

In [ ]:
source_user

In [ ]:
fig = px.bar(source_user,
            x = 'source_id', y = 'cac',
            title = 'стоимость привлечения клиента из каждого источника перехода')
fig.show()

##### установление количества покупателей из  разных девайсов

In [ ]:
# создание таблицы с количеством покупателей из разных устройств
device_count=df_v.pivot_table(index='main_device',values='Uid',aggfunc='count').reset_index()
device_count

In [ ]:
fig = px.bar(device_count,x='main_device', y = 'Uid',color='main_device',
                 title ='количество покупателей с разных устройств')
fig.update_xaxes(title_text='устройства')
fig.update_yaxes(title_text='количество покупателей')
fig.show()

Люди совершают покупки с сенсорных устройств значительно меньше, это может быть связано с тем, что средний чек каждого покупателя составляет несколько тысяч, и решение о таких расходах не принимается "на ходу".

######  Насколько окупились расходы? (ROI) <a name="4.3-bullet"></a>

In [ ]:
df_v.info()

In [ ]:
df_o.info()

 ###### Расчет  ROI по источникам перехода 

In [ ]:
sour_pv_s

In [ ]:
df_o=df_o.merge(sour_pv_s,on='Uid')

In [ ]:
#формирование датасета с доходами из каждого источника перехода
sourse_rev=df_o.pivot_table(index='main_sourse',values='Revenue',aggfunc='sum')

In [ ]:
sourse_rev.reset_index(drop=False, inplace=True)
sourse_rev.columns=['source_id','Revenue']

In [ ]:
sourse_count

In [ ]:
df_с_source

In [ ]:
#создание датасета с затратами и прибылью для каждого  рекламного источника 
rev_cac=df_с_source.merge(sourse_rev,on='source_id')

In [ ]:
rev_cac

In [ ]:
# расчет RIO 
rev_cac['ROI']=(rev_cac['Revenue']-rev_cac['costs'])/rev_cac['costs'] 

In [ ]:
rev_cac

In [ ]:
# построение графика с окупаемостью каждого рекламного источника
fig = px.scatter(rev_cac, x = 'source_id', y = 'ROI',
                 title =' Окупаемость ROI каждого источника перехода')
fig.update_xaxes(title_text='id источника перехода')
fig.update_yaxes(title_text='окупаемость ROI')
fig.show()

Все расходы, за исключением ресурса 3, окупились. Самыми нерентабельными оказались ресурсы с номерами 3,4 и 9,10

Сравним окупаемость с расходами на каждый источник

In [ ]:
fig = px.scatter(df_с_source, x = 'source_id', y = 'costs',
                 title =' Расходы на рекламу на каждый источник покупателей')
fig.update_xaxes(title_text='id источника перехода')
fig.update_yaxes(title_text='расходы на источник перехода')
fig.show()

Соотнесем окупаемость, расходы и количество новых покупателей

In [ ]:
fig = px.scatter(sourse_count, x = 'source_id', y = 'Uid',
                 title =' Количество покупателей с каждого  источника')
fig.update_xaxes(title_text='id источника перехода')
fig.update_yaxes(title_text='количество совершивших заказ пользователей ')
fig.show()

#### Вывод <a name="4.4-bullet"></a>

- расходы на рекламные источники неравномерны
- так же расходам на рекламу характерна сезонность, с весны расходы падают
- расходы на рекламу окупились во всех случаях, за исключением источника номер 3
- рекламный источник с под номером 3 имеет низкую рентабельность, так как на него уходит много денег из-за чего выходит отрицательная окупаемость
- люди делают покупки с сенсорных устройств значительно меньше, это может быть связано с тем, что средний чек каждого покупателя составляет несколько тысяч, и решение о таких расходах не принимается "на ходу".

## Общий вывод <a name="5-bullet"></a>

- люди не склонны совершать покупки в свой первый сеанс 
- основной заработок с когорты приходится на месяц, когда она была сформирована
- воспользовавшись сервисом один раз, малая группа людей будет пользоваться сервисом повторно в течение года, таким образом база пользователей уверенно наращивается
- с весны до лета количество новых пользователей уменьшается, затем осенью количество новых пользователей резко возрастает, точно так же уменьшаются и увеличиваютяс расходы на рекламу по месяцам. Можно предположить, что количество новых пользователей зависит от рекламы. Однако важно отметить, что летом проводится меньше всего мероприятий. Расходы на рекламу летом могут быть минимальны, так как в ней нет смысла
- средний чек не меняется с течением времени
- средний чек и доход от одного покупателя не зависит от сезонности
-  есть смысл снизить расходы на рекламный источник номер 3. Он имеет низкую рентабельность, так как на него уходит много денег, при этом у него отрицательная окупаемость ROI
- есть смысл увеличить расходы на источник номер 4, так как от него приходит большое количество клиентов
- выбирая между рекламой для сенсорных устройств или для стационарных, лучше выбрать рекламу для стационарных устройств, так как с этих устройств приходит большее количество покупателей